In [71]:

import pandas as pd
import numpy as np
from scipy import stats
from scipy.special import boxcox1p
import pickle
from os import path
import statsmodels.api as sm

# data Visualization
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go
# Machine learning algorithms and model evaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.feature_selection import RFE

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Encoding
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [72]:
# Read the Resource files
features = pd.read_csv('../Resources/features.csv')
stores = pd.read_csv("../Resources/stores.csv")
train = pd.read_csv("../Resources/train.csv")
walmart_data = pd.read_csv('../Resources/walmart-sales-dataset-of-45stores.csv')

In [73]:
features.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


In [74]:
features.value_counts()

Store  Date        Temperature  Fuel_Price  MarkDown1  MarkDown2  MarkDown3  MarkDown4  MarkDown5  CPI         Unemployment  IsHoliday
1      2011-11-11  59.11        3.297       10382.90   6115.67    215.07     2406.62    6551.42    217.998085  7.866         False        1
26     2011-12-30  18.80        3.402       4124.23    34472.24   159.04     799.85     2194.68    136.643258  7.598         True         1
       2012-09-07  61.58        3.921       4133.77    6.00       72.78      526.42     3514.48    138.472936  7.405         True         1
       2012-08-31  63.69        3.884       13118.17   3.40       86.31      2176.09    2555.38    138.377194  7.405         False        1
       2012-08-24  62.08        3.874       4507.08    33.60      149.46     1866.46    4685.63    138.281452  7.405         False        1
                                                                                                                                         ..
13     2013-02-15  27.12 

In [75]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [76]:
# Turn the data into a DataFrame
df = pd.DataFrame(walmart_data)

In [77]:
# Get the general statistics for the data.
df.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435.000000,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,23.000000,1.046965e+06,0.069930,60.663782,3.358607,171.578394,7.999151
std,12.988182,5.643666e+05,0.255049,18.444933,0.459020,39.356712,1.875885
min,1.000000,2.099862e+05,0.000000,-2.060000,2.472000,126.064000,3.879000
25%,12.000000,5.533501e+05,0.000000,47.460000,2.933000,131.735000,6.891000
50%,23.000000,9.607460e+05,0.000000,62.670000,3.445000,182.616521,7.874000
75%,34.000000,1.420159e+06,0.000000,74.940000,3.735000,212.743293,8.622000
max,45.000000,3.818686e+06,1.000000,100.140000,4.468000,227.232807,14.313000


In [78]:
# Check for NA values.
na_count = df.isna().sum()
print(na_count)

Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64


In [79]:
# Convert date to datetime 
df["Date"] = pd.to_datetime(df["Date"], format="%d-%m-%Y")
df["Year"] = df["Date"].dt.year
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106,2010
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106,2010
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106,2010
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106,2010
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106,2010
...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.95,0,64.88,3.997,192.013558,8.684,2012
6431,45,2012-10-05,733455.07,0,64.89,3.985,192.170412,8.667,2012
6432,45,2012-10-12,734464.36,0,54.47,4.000,192.327265,8.667,2012
6433,45,2012-10-19,718125.53,0,56.47,3.969,192.330854,8.667,2012


In [80]:
# Create the X dataset without the y variable
X_full = df.drop(columns= ['Weekly_Sales', 'Date'])
X_full.columns


Index(['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Year'],
      dtype='object')

In [81]:
# Show the X dataset
X_full

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year
0,1,0,42.31,2.572,211.096358,8.106,2010
1,1,1,38.51,2.548,211.242170,8.106,2010
2,1,0,39.93,2.514,211.289143,8.106,2010
3,1,0,46.63,2.561,211.319643,8.106,2010
4,1,0,46.50,2.625,211.350143,8.106,2010
...,...,...,...,...,...,...,...
6430,45,0,64.88,3.997,192.013558,8.684,2012
6431,45,0,64.89,3.985,192.170412,8.667,2012
6432,45,0,54.47,4.000,192.327265,8.667,2012
6433,45,0,56.47,3.969,192.330854,8.667,2012


In [82]:
# Create the y dataset
y = df['Weekly_Sales'].values.reshape(-1, 1)
y

array([[1643690.9 ],
       [1641957.44],
       [1611968.17],
       ...,
       [ 734464.36],
       [ 718125.53],
       [ 760281.43]])

In [83]:
# Now split the data into training and testing sets
X_full_train, X_full_test, y_train, y_test = train_test_split(X_full, y, random_state=42)

In [84]:
# Create the models
lr1 = LinearRegression()


# Fit the first model to the full training data. 
lr1.fit(X_full_train, y_train)



LinearRegression()

In [86]:
# Calculate the mean_squared_error and the r-squared value
# for the testing data

# Use our models to make predictions
predicted1 = lr1.predict(X_full_test)

# Score the predictions with mse and r2
mse1 = mean_squared_error(y_test, predicted1)
r21 = r2_score(y_test, predicted1)

print(f"All Features:")
print(f"mean squared error (MSE): {mse1}")
print(f"R-squared (R2): {r21}")



All Features:
mean squared error (MSE): 273140311314.33768
R-squared (R2): 0.14901073544155363
